# Building a YouTube Transcript Summarizer with LangChain and OpenRouter

This tutorial walks through building a system to extract, process and summarize YouTube video transcripts using LangChain, OpenRouter (with Google's Gemini model), and semantic text chunking.

The final output will be a concise summary of the video with key takeaways, processed through semantic chunking and refined using the LLM.

## Key Features

1. **Semantic Chunking**: Instead of simple text splitting, uses semantic understanding to create meaningful chunks
2. **Custom Prompts**: Uses separate prompts for initial summary and refinement
3. **Refinement Chain**: Processes chunks sequentially, refining the summary with each new piece of context
4. **OpenRouter Integration**: Leverages Google's Gemini model through OpenRouter's API

This system provides a sophisticated way to generate meaningful summaries from YouTube video transcripts, making it easier to extract key information from long-form content.

## Step 1: Import necessary libraries

In [3]:
import re
import os
from typing import Optional
from dotenv import load_dotenv

from langchain_core.utils.utils import secret_from_env
from langchain_openai import ChatOpenAI
from langchain.schema import Document
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts import PromptTemplate
from pydantic import Field, SecretStr
from youtube_transcript_api import YouTubeTranscriptApi

## Step 2: YouTube Transcript Extraction
Then, we create functions to extract video transcripts from YouTube URLs:

In [4]:
def extract_video_id(url):
    """
    Extract YouTube video ID from URL.
    Parameters:
        url (str): The YouTube URL to extract the video ID from.
    Returns:
        str: The extracted video ID, or None if no valid ID is found.
    """
    patterns = [
        r'(?:v=|\/)([0-9A-Za-z_-]{11}).*',
        r'(?:youtu\.be\/)([0-9A-Za-z_-]{11})'
    ]
    
    for pattern in patterns:
        match = re.search(pattern, url)
        if match:
            return match.group(1)
    return None

def get_youtube_transcript(url):
    """
    Get the transcript of a YouTube video.
    Parameters:
        url (str): The YouTube URL to get the transcript from.
    Returns:
        str: The transcript of the video, or an error message if the URL is invalid.
    """
    try:
        video_id = extract_video_id(url)
        if not video_id:
            raise ValueError("Invalid YouTube URL")
            
        transcript_list = YouTubeTranscriptApi.get_transcript(video_id)
        full_transcript = ' '.join(entry['text'] for entry in transcript_list)
        return full_transcript #count
        
    except Exception as e:
        return f"Error: {str(e)}"

In [5]:
youtube_url = 'https://www.youtube.com/watch?v=q4DQaMtHvsI&ab_channel=InstituteofPolicyStudies%28IPS%29%2CSingapore'

In [ ]:
text = get_youtube_transcript(youtube_url)
print(text)

## Step 3: Text Embedding with Stella Model
We use the Stella language model for text embeddings:

In [ ]:
#https://huggingface.co/NovaSearch/stella_en_400M_v5/blob/main/README.md
# Define the model name and configuration
model_name = "dunzhang/stella_en_400M_v5"
model_kwargs = {
    'trust_remote_code': True,
    'device': 'cpu',
    'config_kwargs': {
        'use_memory_efficient_attention': False,
        'unpad_inputs': False
    }
}
encode_kwargs = {
    'normalize_embeddings': False
}

# Initialize the HuggingFaceEmbeddings with the Stella model
stella_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs)
document_embeddings = stella_embeddings.embed_documents(text)

In [ ]:
#https://python.langchain.com/api_reference/experimental/text_splitter/langchain_experimental.text_splitter.SemanticChunker.html
chunker = SemanticChunker(embeddings=stella_embeddings)

# SSplit text into chunks and convert to Documents
chunks = chunker.split_text(text)
documents = [Document(page_content=chunk) for chunk in chunks]

for i, doc in enumerate(documents):
    print(f"Document {i + 1}:")
    pprint(doc.page_content)
    print("-" * 50)

## Step 4: Setting up OpenRouter with Gemini
We create a custom ChatOpenRouter class to use OpenRouter's API with Google's Gemini model:

In [11]:
load_dotenv()

# https://openrouter.ai/google/gemini-2.0-flash-exp:free/api
class ChatOpenRouter(ChatOpenAI):
    openai_api_key: Optional[SecretStr] = Field(
        alias="api_key",
        default_factory=secret_from_env("OPENROUTER_API_KEY", default=None),
    )
    @property
    def lc_secrets(self) -> dict[str, str]:
        return {"openai_api_key": "OPENROUTER_API_KEY"}

    def __init__(self,
                 openai_api_key: Optional[str] = None,
                 **kwargs):
        openai_api_key = (
            openai_api_key or os.environ.get("OPENROUTER_API_KEY")
        )
        super().__init__(
            base_url="https://openrouter.ai/api/v1",
            openai_api_key=openai_api_key,
            **kwargs
        )

llm = ChatOpenRouter(
    model_name="google/gemini-2.0-flash-exp:free"
)

## Step 5: Prompting the LLM for Summarization Chain with Refinement
We use the refine summarization chain to generate a summary of the video transcript:


In [ ]:
# Reference https://medium.com/the-data-perspectives/custom-prompts-for-langchain-chains-a780b490c199
question_template = PromptTemplate.from_template("""
Write a concise summary of the following youtube transcript with key takeaways for the audience:
"{text}"
CONCISE SUMMARY:""")

refine_template = PromptTemplate.from_template("""Your job is to produce a final key takeaways summary.
We have provided an existing summary up to a certain point: {existing_answer}

We have the opportunity to refine the existing summary (only if needed) with some more context below.
------------
{text}
------------
Given the new context, refine the original summary with new key takeaways.
If the context isn't useful, return the original summary.\
""")

# Load the refine summarization chain
chain = load_summarize_chain(
    llm,
    chain_type="refine",
    question_prompt=question_template,
    refine_prompt = refine_template,
    verbose=True,
    document_variable_name="text", 
    initial_response_name="existing_answer"
)

## Step 6: Putting It All Together
Here's how to use the complete system:

In [ ]:
output_comb = chain.invoke(documents)
pprint(output_comb)